In [1]:
//json
val people = spark.read.json("people.json")

Intitializing Scala interpreter ...

Spark Web UI available at http://ce7d2534405e:4040
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1597569480518)
SparkSession available as 'spark'


people: org.apache.spark.sql.DataFrame = [age: bigint, name: string]


In [2]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
|  30|Michael|
|  19|    Eva|
|null|   Emma|
+----+-------+



In [3]:
//csv
val people_txt = spark.read.option("inferSchema", "true").csv("people.txt").select(col("_c0").as("name"), col("_c1"))
people_txt.show()

+-------+----+
|   name| _c1|
+-------+----+
|Michael|29.0|
|   Andy|30.0|
| Justin|19.0|
+-------+----+



people_txt: org.apache.spark.sql.DataFrame = [name: string, _c1: double]


### Row collection

In [37]:
// import org.apache.spark.sql._
import spark.implicits._
import org.apache.spark.sql.types._

import spark.implicits._
import org.apache.spark.sql.types._


In [38]:
val person1 = Seq(Row("Greg", 32), Row("ann", 12), Row("Marek", 26))

person1: Seq[org.apache.spark.sql.Row] = List([Greg,32], [ann,12], [Marek,26])


In [40]:
val someSchema = List(
    StructField("name", StringType, true),
    StructField("age", IntegerType, true)
)

someSchema: List[org.apache.spark.sql.types.StructField] = List(StructField(name,StringType,true), StructField(age,IntegerType,true))


In [41]:
val df = spark.createDataFrame(spark.sparkContext.parallelize(person1),
  StructType(someSchema))

df: org.apache.spark.sql.DataFrame = [name: string, age: int]


In [42]:
df.show()

+-----+---+
| name|age|
+-----+---+
| Greg| 32|
|  ann| 12|
|Marek| 26|
+-----+---+



In [224]:
val random = new scala.util.Random(123)
val years = 6
val names = List("Alice", "Betty", "Chris", "Dan", "Greg")
val names_count = names.length

val names_list = (1 to years).flatMap(x => names.map(y => y)).sorted

val year = (1 to names_count).flatMap(x => (0 until years).map(y => 2005 + y))

val starting_salary = (1 to names_count).map(x => {val n = random.nextGaussian()*1000 + 4000; 
                                                   BigDecimal(n).setScale(2, BigDecimal.RoundingMode.HALF_UP
                                                                         ).toDouble})



random: scala.util.Random = scala.util.Random@4c4a9898
years: Int = 6
names: List[String] = List(Alice, Betty, Chris, Dan, Greg)
names_count: Int = 5
names_list: scala.collection.immutable.IndexedSeq[String] = Vector(Alice, Alice, Alice, Alice, Alice, Alice, Betty, Betty, Betty, Betty, Betty, Betty, Chris, Chris, Chris, Chris, Chris, Chris, Dan, Dan, Dan, Dan, Dan, Dan, Greg, Greg, Greg, Greg, Greg, Greg)
year: scala.collection.immutable.IndexedSeq[Int] = Vector(2005, 2006, 2007, 2008, 2009, 2010, 2005, 2006, 2007, 2008, 2009, 2010, 2005, 2006, 2007, 2008, 2009, 2010, 2005, 2006, 2007, 2008, 2009, 2010, 2005, 2006, 2007, 2008, 2009, 2010)
starting_salary: scala.collection.immutable.IndexedSeq[Double] = Vector(2561.95, 4634.2, 4226.06, 4277.46, 4184.32)


In [215]:
var temp_salary = starting_salary.flatMap(x => x :: List.fill(years-1)(0.0)).toList

temp_salary: List[Double] = List(2561.95, 0.0, 0.0, 0.0, 0.0, 0.0, 4634.2, 0.0, 0.0, 0.0, 0.0, 0.0, 4226.06, 0.0, 0.0, 0.0, 0.0, 0.0, 4277.46, 0.0, 0.0, 0.0, 0.0, 0.0, 4184.32, 0.0, 0.0, 0.0, 0.0, 0.0)


In [219]:
def replaceZero(l: List[Double]): List[Double]= {
    var diff = random.nextGaussian()*0.09 + 0.1;
    l match {
      case x :: y :: tail if x != 0.0 & y !=0 => x :: replaceZero(y :: tail)
      case x :: 0.0 :: tail if x != 0.0 => x :: replaceZero((x * (1+diff)):: tail)
      case x :: Nil  if x != 0.0 => x :: Nil
    }
}

def roundToTwoDec(l: List[Double]) = 
            l.map(x => BigDecimal(x).setScale(2, BigDecimal.RoundingMode.HALF_UP).toDouble)

def zipMany[X,Y,Z](xs: Seq[X], ys: Seq[Y], zs: Seq[Z]): Seq[(X,Y,Z)] =
    for (((x, y), z) <- xs zip ys zip zs) yield (x, y, z)

replaceZero: (l: List[Double])List[Double]
roundToTwoDec: (l: List[Double])List[Double]
zipMany: [X, Y, Z](xs: Seq[X], ys: Seq[Y], zs: Seq[Z])Seq[(X, Y, Z)]


In [ ]:
// diffrent approach with mutation of the collection

// for (    
//     n <- 0 until names_count;
//     y <- 0 until (years-1)
// ){
//     var x = random.nextGaussian()*0.09 + 0.1;
//     var index = (years*n +1) +y
    
//     temp_salary(index) =  temp_salary(index - 1) +100
// }
// temp_salary

In [220]:
val salary = roundToTwoDec(replaceZero(temp_salary))

salary: List[Double] = List(2561.95, 2891.88, 3557.24, 4582.31, 5250.59, 5489.16, 4634.2, 4805.67, 4894.54, 5684.75, 6600.54, 6628.51, 4226.06, 3986.92, 3674.72, 3767.04, 3840.6, 4401.84, 4277.46, 4685.72, 5193.04, 6069.9, 7135.37, 8856.8, 4184.32, 4172.95, 4637.61, 4875.82, 5077.4, 5630.17)


In [221]:
val df = zipMany(names_list, year, salary).toDF("name", "year", "salary")

df: org.apache.spark.sql.DataFrame = [name: string, year: int ... 1 more field]


In [222]:
df.show()

+-----+----+-------+
| name|year| salary|
+-----+----+-------+
|Alice|2005|2561.95|
|Alice|2006|2891.88|
|Alice|2007|3557.24|
|Alice|2008|4582.31|
|Alice|2009|5250.59|
|Alice|2010|5489.16|
|Betty|2005| 4634.2|
|Betty|2006|4805.67|
|Betty|2007|4894.54|
|Betty|2008|5684.75|
|Betty|2009|6600.54|
|Betty|2010|6628.51|
|Chris|2005|4226.06|
|Chris|2006|3986.92|
|Chris|2007|3674.72|
|Chris|2008|3767.04|
|Chris|2009| 3840.6|
|Chris|2010|4401.84|
|  Dan|2005|4277.46|
|  Dan|2006|4685.72|
+-----+----+-------+
only showing top 20 rows



In [199]:
import org.apache.spark.sql._

import org.apache.spark.sql._


In [206]:
df.filter(col("name") =!= "Alice" && col("year") > 2007).orderBy(col("salary").desc).show()

+-----+----+-------+
| name|year| salary|
+-----+----+-------+
| Greg|2010|7160.96|
|  Dan|2010|6777.12|
|  Dan|2009|6253.18|
| Greg|2009|6139.79|
|Betty|2010|5854.52|
|Chris|2010|5653.53|
| Greg|2008|5602.12|
|Chris|2009|5495.22|
|  Dan|2008|5104.51|
|Chris|2008|4488.78|
|Betty|2009|4447.36|
|Betty|2008|4183.86|
+-----+----+-------+

